## REBEL: RErank BEyond reLevance
This notebook will walk you through how to set up and run the REBEL method for RAG reranking

In [1]:
# pip install llama_index from our fork, once the PR is merged this will just be a normal llama_index install
# NOTE, restart the session after installing then skip this cell
!git clone https://github.com/bvarjavand/llama_index.git
!cd llama_index && pip install --quiet -e .
!cd llama_index/llama-index-core && pip install --quiet -e .

Cloning into 'llama_index'...
remote: Enumerating objects: 92415, done.
remote: Counting objects: 100% (12947/12947), done.
remote: Compressing objects: 100% (1079/1079), done.
remote: Total 92415 (delta 12181), reused 11871 (delta 11865), pack-reused 79468 (from 1)
Receiving objects: 100% (92415/92415), 226.40 MiB | 38.76 MiB/s, done.
Resolving deltas: 100% (63230/63230), done.
Updating files: 100% (10801/10801), done.

[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-agent-openai 0.4.2 requires llama-index-core<0.13.0,>=0.12.11, but you have llama-index-core 0.12.10 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import os

OPENAI_API_KEY="sk-" # put your openai key here
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
OPENAI_API_BASE="" # optional, if you want to use another hosted endpoint
if OPENAI_API_BASE:
    os.environ["OPENAI_API_BASE"] = OPENAI_API_BASE


In [3]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.postprocessor import LLMRerank
from llama_index.llms.openai import OpenAI
from IPython.display import Markdown, display
from llama_index.core import Settings
import nest_asyncio
nest_asyncio.apply()

if OPENAI_API_BASE:
    Settings.llm = OpenAI(temperature=0, api_key=OPENAI_API_KEY, model="gpt-4o", api_base=OPENAI_API_BASE)
else:
    Settings.llm = OpenAI(temperature=0, api_key=OPENAI_API_KEY, model="gpt-4o")
Settings.chunk_size = 512

## Build RAG index

In [4]:
from pathlib import Path
import requests
from llama_index.embeddings.openai import OpenAIEmbedding

# save documents from wikipedia with these title(s):
wiki_titles = [
    "Vincent van Gogh",
]

# pull documents from wikipedia
data_path = Path("data_wiki")
for title in wiki_titles:
    response = requests.get(
        "https://en.wikipedia.org/w/api.php",
        params={
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            "explaintext": True,
        },
    ).json()
    page = next(iter(response["query"]["pages"].values()))
    wiki_text = page["extract"]

    if not data_path.exists():
        Path.mkdir(data_path)

    with open(data_path / f"{title}.txt", "w") as fp:
        fp.write(wiki_text)

if OPENAI_API_BASE:
    embed_model = OpenAIEmbedding(model="text-embedding-3-large", api_key=OPENAI_API_KEY, api_base=OPENAI_API_BASE)
else:
    embed_model = OpenAIEmbedding(model="text-embedding-3-large", api_key=OPENAI_API_KEY)

# load documents
documents = SimpleDirectoryReader("./data_wiki/").load_data()
# build index
index = VectorStoreIndex.from_documents(
    documents, embed_model=embed_model
)

## Define query string, retrieve nodes, and rerank them

In [16]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import QueryBundle
from llama_index.core.postprocessor import REBELRerank

### You can define any query string you want here ###
query_str = "Which date did Paul Gauguin arrive in Arles?"
query_bundle = QueryBundle(query_str)

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=50,
)

# retrieve nodes
retrieved_nodes = retriever.retrieve(query_bundle)

# configure reranker
if OPENAI_API_BASE:
    reranked = REBELRerank(llm=OpenAI(model='gpt-4o', api_base=OPENAI_API_BASE), top_n=10)
else:
    reranked = REBELRerank(llm=OpenAI(model='gpt-4o'), top_n=10)

# rerank nodes
reranked_nodes = reranked.postprocess_nodes(retrieved_nodes, query_bundle)

In [17]:
import pandas as pd
import torch
from IPython.display import display, HTML

def pretty_print(df):
    return display(HTML(df.to_html().replace("\\n", "<br>")))


def visualize_nodes(nodes) -> None:
    result_dicts = []
    for node in nodes:
        result_dict = {"Score": node.score, "Text": node.node.get_text()}
        result_dicts.append(result_dict)

    pretty_print(pd.DataFrame(result_dicts))

### Top 5 nodes from initial retrieval:

In [18]:
visualize_nodes(retrieved_nodes[:5])

,Score,Text
0,0.531673,"==== Gauguin's visit (1888) ==== When Gauguin agreed to visit Arles in 1888, Van Gogh hoped for friendship and to realise his idea of an artists' collective. Van Gogh prepared for Gauguin's arrival by painting four versions of Sunflowers in one week. ""In the hope of living in a studio of our own with Gauguin,"" he wrote in a letter to Theo, ""I'd like to do a decoration for the studio. Nothing but large Sunflowers."" When Boch visited again, Van Gogh painted a portrait of him, as well as the study The Poet Against a Starry Sky.In preparation for Gauguin's visit, Van Gogh bought two beds on advice from the station's postal supervisor Joseph Roulin, whose portrait he painted. On 17 September, he spent his first night in the still sparsely furnished Yellow House. When Gauguin consented to work and live in Arles with him, Van Gogh started to work on the Décoration for the Yellow House, probably the most ambitious effort he ever undertook. He completed two chair paintings: Van Gogh's Chair and Gauguin's Chair.After much pleading from Van Gogh, Gauguin arrived in Arles on 23 October and, in November, the two painted together. Gauguin depicted Van Gogh in his The Painter of Sunflowers; Van Gogh painted pictures from memory, following Gauguin's suggestion. Among these ""imaginative"" paintings is Memory of the Garden at Etten. Their first joint outdoor venture was at the Alyscamps, when they produced the pendants Les Alyscamps. The single painting Gauguin completed during his visit was his portrait of Van Gogh.Van Gogh and Gauguin visited Montpellier in December 1888, where they saw works by Courbet and Delacroix in the Musée Fabre. Their relationship began to deteriorate; Van Gogh admired Gauguin and wanted to be treated as his equal, but Gauguin was arrogant and domineering, which frustrated Van Gogh."
1,0.507521,"Because the Yellow House had to be furnished before he could fully move in, Van Gogh moved from the Hôtel Carrel to the Café de la Gare on 7 May 1888. He had befriended the Yellow House's proprietors, Joseph and Marie Ginoux, and was able to use it as a studio. Van Gogh wanted a gallery to display his work and started a series of paintings that eventually included Van Gogh's Chair (1888), Bedroom in Arles (1888), The Night Café (1888), Café Terrace at Night (September 1888), Starry Night Over the Rhone (1888), and Still Life: Vase with Twelve Sunflowers (1888), all intended for the decoration for the Yellow House.Van Gogh wrote that with The Night Café he tried ""to express the idea that the café is a place where one can ruin oneself, go mad, or commit a crime"". When he visited Saintes-Maries-de-la-Mer in June, he gave lessons to a Zouave second lieutenant – Paul-Eugène Milliet – and painted boats on the sea and the village. MacKnight introduced Van Gogh to Eugène Boch, a Belgian painter who sometimes stayed in Fontvieille, and the two exchanged visits in July.\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t==== Gauguin's visit (1888) ==== When Gauguin agreed to visit Arles in 1888, Van Gogh hoped for friendship and to realise his idea of an artists' collective. Van Gogh prepared for Gauguin's arrival by painting four versions of Sunflowers in one week. ""In the hope of living in a studio of our own with Gauguin,"" he wrote in a letter to Theo, ""I'd like to do a decoration for the studio. Nothing but large Sunflowers."" When Boch visited again, Van Gogh painted a portrait of him, as well as the study The Poet Against a Starry Sky.In preparation for Gauguin's visit, Van Gogh bought two beds on advice from the station's postal supervisor Joseph Roulin, whose portrait he painted. On 17 September, he spent his first night in the still sparsely furnished Yellow House."
2,0.501718,"Gauguin fled Arles, never to see Van Gogh again. They continued to correspond, and in 1890, Gauguin proposed they form a studio in Antwerp. Meanwhile, other visitors to the hospital included M

### Top 5 nodes from reranking

In [27]:
from IPython.display import Markdown
Markdown(reranked.choice_select_prompt.template)

Inferred Properties:

1. **Historical accuracy (0-5):** Higher if the document provides precise and correct historical details about Paul Gauguin's arrival in Arles. A score of 5 indicates highly accurate and specific information, while a score of 0 indicates vague or incorrect details.

2. **Authoritativeness (0-5):** Higher if the document is sourced from reputable historical texts, art history experts, or well-regarded publications. A score of 5 might be a citation from a recognized art historian or a respected museum publication, while a score of 0 might be an unsourced or dubious claim.

3. **Contextual detail (0-5):** Higher if the document provides additional context about the event, such as the significance of Gauguin's arrival in Arles or related historical events. A score of 5 indicates rich contextual information, while a score of 0 indicates minimal or no context.

Scoring Rubric:

- **Relevance (0-10):** A score of 10 means the document directly addresses the date of Paul Gauguin's arrival in Arles. A score of 0 means it is off-topic.

- **Historical accuracy (0-5):** Assign based on the precision and correctness of the historical details provided.

- **Authoritativeness (0-5):** Assign based on the credibility and source quality.

- **Contextual detail (0-5):** Assign based on the richness of additional context provided.

Weighted Composite Score:

Final Score = Relevance + 0.5*(Historical accuracy) + 0.5*(Authoritativeness) + 0.5*(Contextual detail)

Instructions:

After this prompt, you will see:

Document 1: <summary>  
Document 2: <summary>  
...  
Document N: <summary>  
Question: "Which date did Paul Gauguin arrive in Arles?"

1. Assign Relevance to each document (0-10). Discard documents with Relevance < 3.
2. For remaining documents, assign Historical accuracy, Authoritativeness, and Contextual detail (each 0-5).
3. Compute Final Score as described above.
4. Sort all remaining documents by Final Score (descending).
5. If two documents have identical Final Scores, pick consistently, for example by preferring the one with higher Authoritativeness.
6. If no document remains, output nothing.
7. Output only: Doc: [number], Relevance: [score] for each selected document, no commentary or explanation, where [score] is actually the final score.

"Example format: 
"
"Document 1:
<summary of document 1>

"
"Document 2:
<summary of document 2>

"
"...

"
"Document 10:
<summary of document 10>

"
"Question: <question>
"
"Answer:
"
"Doc: 9, Relevance: 7
"
"Doc: 3, Relevance: 4
"
"Doc: 7, Relevance: 3

"
"Let's try this now: 

"
"{context_str}
"
"Question: {query_str}
"
"Answer:
"

In [28]:
visualize_nodes(reranked_nodes[:5])

,Score,Text
0,12.0,"==== Gauguin's visit (1888) ==== When Gauguin agreed to visit Arles in 1888, Van Gogh hoped for friendship and to realise his idea of an artists' collective. Van Gogh prepared for Gauguin's arrival by painting four versions of Sunflowers in one week. ""In the hope of living in a studio of our own with Gauguin,"" he wrote in a letter to Theo, ""I'd like to do a decoration for the studio. Nothing but large Sunflowers."" When Boch visited again, Van Gogh painted a portrait of him, as well as the study The Poet Against a Starry Sky.In preparation for Gauguin's visit, Van Gogh bought two beds on advice from the station's postal supervisor Joseph Roulin, whose portrait he painted. On 17 September, he spent his first night in the still sparsely furnished Yellow House. When Gauguin consented to work and live in Arles with him, Van Gogh started to work on the Décoration for the Yellow House, probably the most ambitious effort he ever undertook. He completed two chair paintings: Van Gogh's Chair and Gauguin's Chair.After much pleading from Van Gogh, Gauguin arrived in Arles on 23 October and, in November, the two painted together. Gauguin depicted Van Gogh in his The Painter of Sunflowers; Van Gogh painted pictures from memory, following Gauguin's suggestion. Among these ""imaginative"" paintings is Memory of the Garden at Etten. Their first joint outdoor venture was at the Alyscamps, when they produced the pendants Les Alyscamps. The single painting Gauguin completed during his visit was his portrait of Van Gogh.Van Gogh and Gauguin visited Montpellier in December 1888, where they saw works by Courbet and Delacroix in the Musée Fabre. Their relationship began to deteriorate; Van Gogh admired Gauguin and wanted to be treated as his equal, but Gauguin was arrogant and domineering, which frustrated Van Gogh."
1,12.0,"After much pleading from Van Gogh, Gauguin arrived in Arles on 23 October and, in November, the two painted together. Gauguin depicted Van Gogh in his The Painter of Sunflowers; Van Gogh painted pictures from memory, following Gauguin's suggestion. Among these ""imaginative"" paintings is Memory of the Garden at Etten. Their first joint outdoor venture was at the Alyscamps, when they produced the pendants Les Alyscamps. The single painting Gauguin completed during his visit was his portrait of Van Gogh.Van Gogh and Gauguin visited Montpellier in December 1888, where they saw works by Courbet and Delacroix in the Musée Fabre. Their relationship began to deteriorate; Van Gogh admired Gauguin and wanted to be treated as his equal, but Gauguin was arrogant and domineering, which frustrated Van Gogh. They often quarrelled; Van Gogh increasingly feared that Gauguin was going to desert him, and the situation, which Van Gogh described as one of ""excessive tension"", rapidly headed towards crisis point.\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t==== Hospital in Arles (December 1888) ====The exact sequence that led to the mutilation of Van Gogh's ear is not known. Gauguin said, fifteen years later, that the night followed several instances of physically threatening behaviour. Their relationship was complex and Theo may have owed money to Gauguin, who suspected the brothers were exploiting him financially. It seems likely that Vincent realised that Gauguin was planning to leave. The following days saw heavy rain, leading to the two men being shut in the Yellow House. Gauguin recalled that Van Gogh followed him after he left for a walk and ""rushed towards me, an open razor in his hand"". This account is uncorroborated; Gauguin was almost certainly absent from the Yellow House that night, most likely staying in a hotel.After an altercation on the evening of 23 December 1888, Van Gogh returned to his room where he seemingly heard voices and either wholly or in part severed his left ear with a razor causing severe bleeding."
2,6.0,"Later he drifted into ill-healt